# In this notebook, we're going to cover some of the most fundamental concepts of tensors using TensorFlow

More specifically, we're going to cover:
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & Numpy
* Using @tf.function (a way to speed up regular Python functions)
* Using GPUs with TensorFlow (or TPU)
* Excecises to try for yourself!

## Introduction to Tensors

In [3]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [2]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [3]:
# Check number of dimensions of a tensor (ndim stands for number of dimensions)
scalar.ndim

0

In [4]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10])>

In [5]:
# Check the dimension of our vector
vector.ndim

1

In [6]:
# Create a matrix (has more than 1 dimension)
matrix = tf.constant([[10, 7], [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]])>

In [7]:
# Ceck the dimension of our matrix
matrix.ndim

2

In [8]:
# Create another matrix
another_matrix = tf.constant([[10., 7.], [3., 2.], [8., 9.]], dtype=tf.float16)
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [9]:
# What's the dimensions of another_matrix?
another_matrix.ndim

2

In [10]:
# Let's create a tensor
tensor = tf.constant([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]], [[13, 14, 15], [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]])>

In [11]:
# What's the dimensions of tensor
tensor.ndim

3

What we've created so far:

* Scalar: a single number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (where n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)

## Creating tensors with tf.Variable

In [12]:
# Create the same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7])>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7])>)

In [13]:
# Let's try change one of the elements in our changeable tensor
changeable_tensor[0] = 7
changeable_tensor

TypeError: 'ResourceVariable' object does not support item assignment

In [19]:
# How about we try .assign()
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7])>

In [21]:
# Let's try change our unchangeable tensor
unchangeable_tensor[0].assign(7)
unchangeable_tensor

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

> 🔑 **Note**: Rarely in practice will you need to decide wheter to use `tf.constant` or `tf.Varaible` to create tensors, as TensorFlow does this for you. However, if in doubt, use `tf.constant` and change it later if needed.

## Creating random tensors
Random tensors are tensors of some arbitrary size which contain random numbers.

In [3]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_1 = random_1.normal(shape=(3, 2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3, 2))

# Are they equal?

random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

## Shuffle the order of elements in a tensor

In [4]:
# Shuffle a tensor (valuable for when you want to shuffle your data so the inherent order doesn't effect learning)
not_shuffled = tf.constant([[10, 7], [3, 4], [2, 5]])
not_shuffled.ndim

2

In [240]:
# Shuffle our non-shuffled tensor
tf.random.set_seed(42)
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]])>

**Exercise:** Read through TensorFlow documentation on random seed generation:<br>
https://www.tensorflow.org/api_docs/python/tf/random/set_seed and practice writing 5 random tensors and shuffle them.


In [206]:
random_t_1 = tf.random.Generator.from_non_deterministic_state().normal(shape=(3, 2))
random_t_2 = tf.random.Generator.from_seed(42).normal(shape=(3, 2))
random_t_3 = tf.random.Generator.from_seed(42, alg="philox").normal(shape=(3, 2))
random_t_4 = tf.random.Generator.from_seed(42, alg="philox").uniform(shape=(3, 2))
random_t_5 = tf.random.Generator.from_seed(42, alg="philox").normal(shape=(3, 2))
tf.random.shuffle(random_t_1), tf.random.shuffle(random_t_2, seed=7), tf.random.shuffle(random_t_3), tf.random.shuffle(random_t_4, seed=24), random_t_5

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[ 1.9927605 ,  0.36401126],
        [ 0.4556108 , -0.89363164],
        [ 0.70621437, -0.32751867]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [-0.23193763, -1.8107855 ],
        [ 0.07595026, -1.2573844 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ],
        [-0.7565803 , -0.06854702]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[0.1889317 , 0.52027524],
        [0.7493447 , 0.73561966],
        [0.45230794, 0.49039817]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>)

## Other ways to make tensors

In [241]:
# Create a tensor of all ones
tf.ones([10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [244]:
# Create a tensor of all zeroes
tf.zeros(shape=(10, 7), dtype=tf.int32)

<tf.Tensor: shape=(10, 7), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]])>

### Turn numpys into tensors<br>
The main difference between NumPy and TensorFlow tensors is that tensors can be run on a GPU (much faster for numerical computing)

In [57]:
# You can also turn NumPy arrays into tensors
import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32) # create a NumPy array between 1 and 25
numpy_A

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])

In [250]:
A = tf.constant(numpy_A, shape=(2, 3, 4))
B = tf.constant(numpy_A)
C = tf.constant(numpy_A, shape=(3, 8))
A, B, C

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]])>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24])>,
 <tf.Tensor: shape=(3, 8), dtype=int32, numpy=
 array([[ 1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16],
        [17, 18, 19, 20, 21, 22, 23, 24]])>)

## Getting information from tensors
When dealing with tensors you probably want to be aware of the following attributes:
* Shape
* Rank
* Axis or dimension
* Size

In [256]:
# Create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros(shape=[2, 3, 4, 5])
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [258]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [265]:
# Get various attributes of our tensor
print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank):", rank_4_tensor.ndim)
print("Shape of tensor:", rank_4_tensor.shape)
print("Elements along the 0 axis:", rank_4_tensor.shape[0])
print("Elements along the last axis:", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor:", tf.size(rank_4_tensor))
print("Total number of elements in our tensor:", tf.size(rank_4_tensor).numpy())

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape of tensor: (2, 3, 4, 5)
Elements along the 0 axis: 2
Elements along the last axis: 5
Total number of elements in our tensor: tf.Tensor(120, shape=(), dtype=int32)
Total number of elements in our tensor: 120


## Indexing tensors
Tensors can be indexed just like Python lists.

In [273]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [281]:
# Get first of each dimension from each index except for the final one
rank_4_tensor[:1, :1, :1, :]
rank_4_tensor[:1, :1, :1]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [285]:
# Create a rank 2 tensor (2 dimensions)
rank_2_tensor = tf.constant([[10, 7], [3, 4]])
rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([2, 2]), 2)

In [286]:
# Get the last item of each of our rank 2 tensor
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4])>

In [287]:
# Add in extra dimension to our rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]])>

In [288]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1)

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]])>

In [289]:
# Expand the 0-axis
tf.expand_dims(rank_2_tensor, axis=0)

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[10,  7],
        [ 3,  4]]])>

## Manipulating tensors (tensor operations)
**Basic operations**
`+`, `-`, `*`, `/`

In [290]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10, 7], [3, 4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]])>

In [291]:
# Multiplication also works
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]])>

In [294]:
# We can use the tensorflow built-in function too
tf.multiply(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]])>

In [295]:
tf.math.multiply(tensor, 11)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[110,  77],
       [ 33,  44]])>

## Matrix multiplication
In machine learning, matrix multiplication is one of the most common tensor operations.<br><br>
There are two rules our tensors (or matrices) need to fulfil if we're going to matrix multiply them:

1. The inner dimensions must match
2. The resulting matrix has the shape of the inner dimensions

In [16]:
# Matrix multiplication in TensorFlow
tensor = tf.constant([[10, 7], [3, 4]])
print(tensor)
tf.linalg.matmul(tensor, tensor)

tf.Tensor(
[[10  7]
 [ 3  4]], shape=(2, 2), dtype=int32)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]])>

In [17]:
a = tf.constant([[1, 2, 5], [7, 2, 1], [3, 3, 3]])
b = tf.constant([[3, 5], [6, 7], [1, 8]])
tf.linalg.matmul(a, b)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[20, 59],
       [34, 57],
       [30, 60]])>

In [18]:
# Matrix multiplication with Python operator "@"
tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]])>

In [19]:
# Create a tensor (3, 2) tensor
X = tf.constant([[1, 2], [3, 4], [5, 6]])
# Create another (3, 2) tensor
Y = tf.constant([[7, 8], [9, 10], [11, 12]])
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]])>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]])>)

In [22]:
# Try to matrix multiply tensors of same shape
X @ Y

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]])>

In [23]:
# Let's change the shape of Y
tf.reshape(Y, (2, 3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  8,  9],
       [10, 11, 12]])>

In [24]:
# Try to multiply X by reshaped Y
X @ tf.reshape(Y, (2, 3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]])>

In [25]:
# What if we reshape X instead
tf.reshape(X, shape=(2, 3)) @ Y

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 58,  64],
       [139, 154]])>

In [26]:
# Can we do the same with transpose
X, tf.transpose(X), tf.reshape(X, shape=(2, 3))

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]])>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]])>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]])>)

In [27]:
# Try matrix multiplication with transpose rather than reshape
tf.matmul(tf.transpose(X), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]])>

**Resource:** Info and example of matrix multiplication: https://www.mathsisfun.com/algebra/matrix-multiplying.html

**The dot product**
Matrix multiplication is also referred to as the dot product.<br>

You can perform matrix multiplication usin:
*  `tf.matmul()`
*  `tf.tensordot()`

In [29]:
# Perofrm the dot product on X and Y (requires X or Y to be transposed)
tf.tensordot(tf.transpose(X), Y, axes=1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]])>

In [30]:
# Perform matrix multiplication between X and Y (transposed)
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]])>

In [31]:
# Perform matrix multiplication between X and Y (reshaped)
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]])>

In [36]:
# Check the values of Y, reshaped Y and transposed Y
print("Normal Y:", Y, "\n")
print("Y reshaped to (2, 3):", tf.reshape(Y, shape=(2, 3)), "\n")
print("Y transposed:", tf.transpose(Y), "\n")

Normal Y: tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32) 

Y reshaped to (2, 3): tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32) 

Y transposed: tf.Tensor(
[[ 7  9 11]
 [ 8 10 12]], shape=(2, 3), dtype=int32) 



Generally, when performing matrix multiplaction on two tensors and one of the axes doesn't line up you will transpose (rather than reshape) one of the tensors to satisfy the matrix multiplaction rules.

## Changing the datatype of a tensor

In [42]:
# Create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])
B.dtype

tf.float32

In [38]:
C = tf.constant([7, 10])
C.dtype

tf.int32

In [43]:
# Change from float32 to float16 (reduced precision)
D = tf.cast(B, dtype=tf.float16)
D.dtype

tf.float16

In [45]:
# Change from int32 to float32
E = tf.cast(C, dtype=tf.float32)
E

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 7., 10.], dtype=float32)>

## Aggregating tensors
Aggregating tensors = condensing them from multiple values down to a smaller amount of values.

In [46]:
# Get the absolute values
D = tf.constant([-7, -10])
tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10])>

Let's go through the following forms of aggregation:
* Get the minimum
* Get the maximum
* Get the mean of a tensor
* Get the sum of a tensor

In [49]:
F = tf.constant([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

In [53]:
# Get the minimum
tf.math.reduce_min(F).numpy()

1

In [52]:
# Get the maximum
tf.math.reduce_max(F).numpy()

9

In [54]:
# Get the mean
tf.math.reduce_mean(F).numpy()

5

In [55]:
# Get the sum
tf.math.reduce_sum(F).numpy()

45

**Excercise:** With what we've just learned, find the variance and standard deviation of our `E` tensor using TensorFlow methods.

In [76]:
np.random.seed(42)
E = tf.constant(np.random.randint(0, 100, size=50))
E

<tf.Tensor: shape=(50,), dtype=int32, numpy=
array([51, 92, 14, 71, 60, 20, 82, 86, 74, 74, 87, 99, 23,  2, 21, 52,  1,
       87, 29, 37,  1, 63, 59, 20, 32, 75, 57, 21, 88, 48, 90, 58, 41, 91,
       59, 79, 14, 61, 61, 46, 61, 50, 54, 63,  2, 50,  6, 20, 72, 38])>

In [79]:
# Find the variance
tf.math.reduce_variance(tf.cast(E, dtype=tf.float16)).numpy()

776.5

In [84]:
import tensorflow_probability as tfp
tfp.stats.variance(E).numpy()

776

In [80]:
# Find the standard deviation
tf.math.reduce_std(tf.cast(E, dtype=tf.float16)).numpy()

27.86

In [81]:
tf.math.reduce_mean(E).numpy()

50

## Find the positional maximum and minimum

In [92]:
# Find the positional maximum and value
tf.math.argmax(E).numpy(), E[tf.math.argmax(E)].numpy(), tf.reduce_max(E).numpy()

(11, 99, 99)

In [93]:
# Find the positional minimum and value
tf.math.argmin(E).numpy(), E[tf.math.argmin(E)].numpy(), tf.reduce_min(E).numpy()

(16, 1, 1)

## Squeezing a tensor (removing all single dimensions)

In [95]:
# Create a tensor to get started
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1, 1, 1, 1, 50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.70362866, 0.5693954 , 0.21715295, 0.09898746, 0.9741869 ,
           0.04486942, 0.82212555, 0.04484725, 0.73791444, 0.10259283,
           0.57590365, 0.59727657, 0.8630384 , 0.66040754, 0.05492282,
           0.15802407, 0.49131155, 0.19955897, 0.67458165, 0.63931715,
           0.12048542, 0.76256716, 0.45332968, 0.69711494, 0.24075091,
           0.57000935, 0.5894742 , 0.49703455, 0.40253878, 0.8912879 ,
           0.5828428 , 0.72290015, 0.42292762, 0.76623964, 0.29939497,
           0.218938  , 0.27585137, 0.24625695, 0.9878806 , 0.60163355,
           0.684584  , 0.8082118 , 0.16198683, 0.16690421, 0.23370552,
           0.9550992 , 0.872218  , 0.16131759, 0.61385024, 0.8296901 ]]]]],
      dtype=float32)>

In [96]:
G_squeezed = tf.squeeze(G)
G_squeezed, G_squeezed.shape

(<tf.Tensor: shape=(50,), dtype=float32, numpy=
 array([0.70362866, 0.5693954 , 0.21715295, 0.09898746, 0.9741869 ,
        0.04486942, 0.82212555, 0.04484725, 0.73791444, 0.10259283,
        0.57590365, 0.59727657, 0.8630384 , 0.66040754, 0.05492282,
        0.15802407, 0.49131155, 0.19955897, 0.67458165, 0.63931715,
        0.12048542, 0.76256716, 0.45332968, 0.69711494, 0.24075091,
        0.57000935, 0.5894742 , 0.49703455, 0.40253878, 0.8912879 ,
        0.5828428 , 0.72290015, 0.42292762, 0.76623964, 0.29939497,
        0.218938  , 0.27585137, 0.24625695, 0.9878806 , 0.60163355,
        0.684584  , 0.8082118 , 0.16198683, 0.16690421, 0.23370552,
        0.9550992 , 0.872218  , 0.16131759, 0.61385024, 0.8296901 ],
       dtype=float32)>,
 TensorShape([50]))

## One-hot encoding tensors

In [98]:
# Create a list of indices
some_list = [0, 1, 2, 3] # could be red, green, blue, purple
tf.one_hot(some_list, depth=4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [103]:
some_other_list = [0, 3, 1, 2]
tf.one_hot(some_other_list, depth=6, on_value="A", off_value="B")

<tf.Tensor: shape=(4, 6), dtype=string, numpy=
array([[b'A', b'B', b'B', b'B', b'B', b'B'],
       [b'B', b'B', b'B', b'A', b'B', b'B'],
       [b'B', b'A', b'B', b'B', b'B', b'B'],
       [b'B', b'B', b'A', b'B', b'B', b'B']], dtype=object)>

## Squaring, log, square root

In [104]:
# Create a new tensor
H = tf.range(1, 10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9])>

In [105]:
# Square it
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81])>

In [108]:
# Find the square root (method requires non-int type)
tf.sqrt(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.99999994, 1.4142134 , 1.7320508 , 1.9999999 , 2.236068  ,
       2.4494896 , 2.6457512 , 2.8284268 , 3.        ], dtype=float32)>

In [110]:
# Find log (method requires non-int type)
tf.math.log(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

## Tensors and NumPy
TensorFlow interacts beautifully with NumPy arrays.<br>
** Note: ** One of the main differences between a TensorFlow tensor and a NumPy array is that a TensorFlow tensor can be run on a GPU or TPU (for faster numerical processing)

In [111]:
# Create a tensor directly from a NumPy array
J = tf.constant(np.array([3., 7., 10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [112]:
# Convert our tensor back to a NumPy array
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [113]:
# Convert tensor J to a NumPy array
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [114]:
# The default types of each are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])
# Check the datatype of each
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

## Finding access to GPUs

In [4]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [1]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3515320909445764270
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6961823744
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4191988365899223693
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
